# 리트리버 성능평가

In [23]:
import pandas as pd
import os
from copy import deepcopy

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)

In [24]:
import ast
import re

def preprocess(text):
    text = re.sub(r"\n", " ", text)
    text = re.sub(r"\\n", " ", text)
    text = re.sub(r"#", " ", text)
    text = re.sub(
        r"[^A-Za-z0-9가-힣.?!,()~‘’“”" ":%&《》〈〉''㈜·\-'+\s一-龥サマーン]",
        "",
        text,
    )  # サマーン 는 predictions.json에 있었음
    text = re.sub(r"\s+", " ", text).strip()  # 두 개 이상의 연속된 공백을 하나로 치환
    # text = re.sub(r"[^A-Za-z0-9가-힣.?!,()~‘’“”"":%&《》〈〉''㈜·\-\'+\s一-龥]", "", text)

    return text


def evaluate_retriever(df, topk=100, is_preprocess=False, is_title=False):
    df["context"] = df["context"].apply(
        lambda x: ast.literal_eval(x)
    )
    correct_ids = []
    wrong_ids = []
    for idx,row in df.iterrows():
        org_context = (
            preprocess(row["original_context"])
            if is_preprocess
            else row["original_context"]
        )
        if is_title:
            for passage in row["context"][:topk]:
                if passage.split(": ")[1] == org_context:
                    correct_ids.append(row["id"])
                    break
            else:
                wrong_ids.append(row["id"])
        else:
            if org_context in row["context"][:topk]:
                correct_ids.append(row["id"])
            else:
                wrong_ids.append(row["id"])

    print(
        f"전체: {len(df)}, 맞은 개수: {len(correct_ids)}, 틀린 개수: {len(wrong_ids)}, 정답률: {(len(correct_ids)/len(df)):.4%}"
    )
    return correct_ids, wrong_ids

## Elasticsearch

### Train

In [6]:
es_path = os.path.join(parent_dir, "data", "es_train")
df_train_es_100 = pd.read_csv(os.path.join(es_path, "train_es_train_100.csv"))
dfs = []
for i in range(10):
    dfs.append(deepcopy(df_train_es_100))

In [7]:
_, _ = evaluate_retriever(deepcopy(df_train_es_100), topk=5, is_preprocess=True)

전체: 3952, 맞은 개수: 3411, 틀린 개수: 541, 정답률: 86.3107%


In [156]:
for df, topk in zip(deepcopy(dfs), [k for k in range(10, 110, 10)]):
    _, _ = evaluate_retriever(df, topk=topk, is_preprocess=True)

전체: 3952, 맞은 개수: 3543, 틀린 개수: 409, 정답률: 89.6508%
전체: 3952, 맞은 개수: 3667, 틀린 개수: 285, 정답률: 92.7885%
전체: 3952, 맞은 개수: 3722, 틀린 개수: 230, 정답률: 94.1802%
전체: 3952, 맞은 개수: 3741, 틀린 개수: 211, 정답률: 94.6609%
전체: 3952, 맞은 개수: 3764, 틀린 개수: 188, 정답률: 95.2429%
전체: 3952, 맞은 개수: 3783, 틀린 개수: 169, 정답률: 95.7237%
전체: 3952, 맞은 개수: 3793, 틀린 개수: 159, 정답률: 95.9767%
전체: 3952, 맞은 개수: 3803, 틀린 개수: 149, 정답률: 96.2298%
전체: 3952, 맞은 개수: 3810, 틀린 개수: 142, 정답률: 96.4069%
전체: 3952, 맞은 개수: 3817, 틀린 개수: 135, 정답률: 96.5840%


### Validation

In [8]:
es_path = os.path.join(parent_dir, "data", "es_train")
df_val_es_100 = pd.read_csv(os.path.join(es_path, "train_es_val_100.csv"))
dfs = []
for i in range(10):
    dfs.append(deepcopy(df_val_es_100))

In [9]:
_, _ = evaluate_retriever(deepcopy(df_val_es_100), topk=5, is_preprocess=True)

전체: 240, 맞은 개수: 209, 틀린 개수: 31, 정답률: 87.0833%


In [5]:
for df, topk in zip(deepcopy(dfs), [k for k in range(10, 110, 10)]):
    _, _ = evaluate_retriever(df, topk=topk, is_preprocess=True)

전체: 240, 맞은 개수: 222, 틀린 개수: 18, 정답률: 92.5000%
전체: 240, 맞은 개수: 227, 틀린 개수: 13, 정답률: 94.5833%
전체: 240, 맞은 개수: 231, 틀린 개수: 9, 정답률: 96.2500%
전체: 240, 맞은 개수: 232, 틀린 개수: 8, 정답률: 96.6667%
전체: 240, 맞은 개수: 232, 틀린 개수: 8, 정답률: 96.6667%
전체: 240, 맞은 개수: 234, 틀린 개수: 6, 정답률: 97.5000%
전체: 240, 맞은 개수: 234, 틀린 개수: 6, 정답률: 97.5000%
전체: 240, 맞은 개수: 234, 틀린 개수: 6, 정답률: 97.5000%
전체: 240, 맞은 개수: 235, 틀린 개수: 5, 정답률: 97.9167%
전체: 240, 맞은 개수: 235, 틀린 개수: 5, 정답률: 97.9167%


### Train + Validation

In [160]:
df_concat = pd.concat([df_train_es_100,df_val_es_100],axis=0)
dfs = []
for i in range(10):
    dfs.append(deepcopy(df_concat))

In [161]:
for df, topk in zip(deepcopy(dfs), [k for k in range(10, 110, 10)]):
    _, _ = evaluate_retriever(df, topk=topk, is_preprocess=True)

전체: 4192, 맞은 개수: 3765, 틀린 개수: 427, 정답률: 89.8139%
전체: 4192, 맞은 개수: 3894, 틀린 개수: 298, 정답률: 92.8912%
전체: 4192, 맞은 개수: 3953, 틀린 개수: 239, 정답률: 94.2987%
전체: 4192, 맞은 개수: 3973, 틀린 개수: 219, 정답률: 94.7758%
전체: 4192, 맞은 개수: 3996, 틀린 개수: 196, 정답률: 95.3244%
전체: 4192, 맞은 개수: 4017, 틀린 개수: 175, 정답률: 95.8254%
전체: 4192, 맞은 개수: 4027, 틀린 개수: 165, 정답률: 96.0639%
전체: 4192, 맞은 개수: 4037, 틀린 개수: 155, 정답률: 96.3025%
전체: 4192, 맞은 개수: 4045, 틀린 개수: 147, 정답률: 96.4933%
전체: 4192, 맞은 개수: 4052, 틀린 개수: 140, 정답률: 96.6603%


## BM25

### Train

In [10]:
bm25_path = os.path.join(parent_dir, "data", "bm25_train")

df_train_bm25_100 = pd.read_csv(os.path.join(bm25_path, "train_bm25_train_100.csv"))
dfs = []
for i in range(10):
    dfs.append(deepcopy(df_train_bm25_100))

In [7]:
for df, topk in zip(deepcopy(dfs), [k for k in range(10, 110, 10)]):
    _, _ = evaluate_retriever(df, topk=topk, is_title=True)

전체: 3952, 맞은 개수: 3317, 틀린 개수: 635, 정답률: 83.9322%
전체: 3952, 맞은 개수: 3448, 틀린 개수: 504, 정답률: 87.2470%
전체: 3952, 맞은 개수: 3499, 틀린 개수: 453, 정답률: 88.5374%
전체: 3952, 맞은 개수: 3529, 틀린 개수: 423, 정답률: 89.2966%
전체: 3952, 맞은 개수: 3557, 틀린 개수: 395, 정답률: 90.0051%
전체: 3952, 맞은 개수: 3577, 틀린 개수: 375, 정답률: 90.5111%
전체: 3952, 맞은 개수: 3591, 틀린 개수: 361, 정답률: 90.8654%
전체: 3952, 맞은 개수: 3599, 틀린 개수: 353, 정답률: 91.0678%
전체: 3952, 맞은 개수: 3611, 틀린 개수: 341, 정답률: 91.3715%
전체: 3952, 맞은 개수: 3616, 틀린 개수: 336, 정답률: 91.4980%


In [3]:
bm25_path = os.path.join(parent_dir, "data", "bm25_train")

df_train_bm25_100 = pd.read_csv(os.path.join(bm25_path, "train_bm25_train_100_notitle.csv"))
dfs = []
for i in range(10):
    dfs.append(deepcopy(df_train_bm25_100))

In [12]:
df_train_bm25_100 = pd.read_csv(os.path.join(bm25_path, "train_bm25_train_100_notitle.csv"))
_, _ = evaluate_retriever(deepcopy(df_train_bm25_100), topk=5)

전체: 3952, 맞은 개수: 3280, 틀린 개수: 672, 정답률: 82.9960%


In [5]:
for df, topk in zip(deepcopy(dfs), [k for k in range(10, 110, 10)]):
    _, _ = evaluate_retriever(df, topk=topk, is_title=False)

전체: 3952, 맞은 개수: 3454, 틀린 개수: 498, 정답률: 87.3988%
전체: 3952, 맞은 개수: 3592, 틀린 개수: 360, 정답률: 90.8907%
전체: 3952, 맞은 개수: 3652, 틀린 개수: 300, 정답률: 92.4089%
전체: 3952, 맞은 개수: 3681, 틀린 개수: 271, 정답률: 93.1427%
전체: 3952, 맞은 개수: 3712, 틀린 개수: 240, 정답률: 93.9271%
전체: 3952, 맞은 개수: 3729, 틀린 개수: 223, 정답률: 94.3573%
전체: 3952, 맞은 개수: 3746, 틀린 개수: 206, 정답률: 94.7874%
전체: 3952, 맞은 개수: 3758, 틀린 개수: 194, 정답률: 95.0911%
전체: 3952, 맞은 개수: 3766, 틀린 개수: 186, 정답률: 95.2935%
전체: 3952, 맞은 개수: 3773, 틀린 개수: 179, 정답률: 95.4706%


### Validation

In [8]:
df_val_bm25_100 = pd.read_csv(os.path.join(bm25_path, "train_bm25_val_100.csv"))
dfs = []
for i in range(10):
    dfs.append(deepcopy(df_val_bm25_100))

In [9]:
for df, topk in zip(deepcopy(dfs), [k for k in range(10, 110, 10)]):
    _, _ = evaluate_retriever(df, topk=topk, is_title=True)

전체: 240, 맞은 개수: 203, 틀린 개수: 37, 정답률: 84.5833%
전체: 240, 맞은 개수: 210, 틀린 개수: 30, 정답률: 87.5000%
전체: 240, 맞은 개수: 215, 틀린 개수: 25, 정답률: 89.5833%
전체: 240, 맞은 개수: 216, 틀린 개수: 24, 정답률: 90.0000%
전체: 240, 맞은 개수: 217, 틀린 개수: 23, 정답률: 90.4167%
전체: 240, 맞은 개수: 217, 틀린 개수: 23, 정답률: 90.4167%
전체: 240, 맞은 개수: 217, 틀린 개수: 23, 정답률: 90.4167%
전체: 240, 맞은 개수: 217, 틀린 개수: 23, 정답률: 90.4167%
전체: 240, 맞은 개수: 218, 틀린 개수: 22, 정답률: 90.8333%
전체: 240, 맞은 개수: 218, 틀린 개수: 22, 정답률: 90.8333%


In [14]:
df_val_bm25_100 = pd.read_csv(os.path.join(bm25_path, "train_bm25_val_100_notitle.csv"))
dfs = []
for i in range(10):
    dfs.append(deepcopy(df_val_bm25_100))

for df, topk in zip(deepcopy(dfs), [k for k in range(10, 110, 10)]):
    _, _ = evaluate_retriever(df, topk=topk, is_title=False)

전체: 240, 맞은 개수: 216, 틀린 개수: 24, 정답률: 90.0000%
전체: 240, 맞은 개수: 223, 틀린 개수: 17, 정답률: 92.9167%
전체: 240, 맞은 개수: 228, 틀린 개수: 12, 정답률: 95.0000%
전체: 240, 맞은 개수: 228, 틀린 개수: 12, 정답률: 95.0000%
전체: 240, 맞은 개수: 228, 틀린 개수: 12, 정답률: 95.0000%
전체: 240, 맞은 개수: 230, 틀린 개수: 10, 정답률: 95.8333%
전체: 240, 맞은 개수: 230, 틀린 개수: 10, 정답률: 95.8333%
전체: 240, 맞은 개수: 230, 틀린 개수: 10, 정답률: 95.8333%
전체: 240, 맞은 개수: 231, 틀린 개수: 9, 정답률: 96.2500%
전체: 240, 맞은 개수: 232, 틀린 개수: 8, 정답률: 96.6667%


In [13]:
df_val_bm25_100 = pd.read_csv(os.path.join(bm25_path, "train_bm25_val_100_notitle.csv"))
_, _ = evaluate_retriever(deepcopy(df_val_bm25_100), topk=5)

전체: 240, 맞은 개수: 205, 틀린 개수: 35, 정답률: 85.4167%


### Train + Validatation

In [10]:
df_concat = pd.concat([df_train_bm25_100, df_val_bm25_100], axis=0)
dfs = []
for i in range(10):
    dfs.append(deepcopy(df_concat))

In [11]:
for df, topk in zip(deepcopy(dfs), [k for k in range(10, 110, 10)]):
    _, _ = evaluate_retriever(df, topk=topk, is_title=True)

전체: 4192, 맞은 개수: 3520, 틀린 개수: 672, 정답률: 83.9695%
전체: 4192, 맞은 개수: 3658, 틀린 개수: 534, 정답률: 87.2615%
전체: 4192, 맞은 개수: 3714, 틀린 개수: 478, 정답률: 88.5973%
전체: 4192, 맞은 개수: 3745, 틀린 개수: 447, 정답률: 89.3368%
전체: 4192, 맞은 개수: 3774, 틀린 개수: 418, 정답률: 90.0286%
전체: 4192, 맞은 개수: 3794, 틀린 개수: 398, 정답률: 90.5057%
전체: 4192, 맞은 개수: 3808, 틀린 개수: 384, 정답률: 90.8397%
전체: 4192, 맞은 개수: 3816, 틀린 개수: 376, 정답률: 91.0305%
전체: 4192, 맞은 개수: 3829, 틀린 개수: 363, 정답률: 91.3406%
전체: 4192, 맞은 개수: 3834, 틀린 개수: 358, 정답률: 91.4599%


## Rerank-Elasticsearch

In [16]:
es_path = os.path.join(parent_dir, "data", "es_train")

df_es_20 = pd.read_csv(os.path.join(es_path, "not_train_es_20_rerank_5.csv"))
df_es_30 = pd.read_csv(os.path.join(es_path, "not_train_es_30_rerank_5.csv"))
df_es_40 = pd.read_csv(os.path.join(es_path, "not_train_es_40_rerank_5.csv"))
df_es_50 = pd.read_csv(os.path.join(es_path, "not_train_es_50_rerank_5.csv"))
df_es_60 = pd.read_csv(os.path.join(es_path, "not_train_es_60_rerank_5.csv"))
df_es_70 = pd.read_csv(os.path.join(es_path, "not_train_es_70_rerank_5.csv"))
df_es_80 = pd.read_csv(os.path.join(es_path, "not_train_es_80_rerank_5.csv"))
df_es_90 = pd.read_csv(os.path.join(es_path, "not_train_es_90_rerank_5.csv"))
df_es_100 = pd.read_csv(os.path.join(es_path, "not_train_es_100_rerank_5.csv"))

dfs = [
    df_es_20,
    df_es_30,
    df_es_40,
    df_es_50,
    df_es_60,
    df_es_70,
    df_es_80,
    df_es_90,
    df_es_100,
]

In [128]:
for df in deepcopy(dfs):
    _, _ = evaluate_retriever(df, topk=5, is_preprocess=False)

전체: 240, 맞은 개수: 220, 틀린 개수: 20, 정답률: 91.6667%
전체: 240, 맞은 개수: 223, 틀린 개수: 17, 정답률: 92.9167%
전체: 240, 맞은 개수: 222, 틀린 개수: 18, 정답률: 92.5000%
전체: 240, 맞은 개수: 222, 틀린 개수: 18, 정답률: 92.5000%
전체: 240, 맞은 개수: 224, 틀린 개수: 16, 정답률: 93.3333%
전체: 240, 맞은 개수: 222, 틀린 개수: 18, 정답률: 92.5000%
전체: 240, 맞은 개수: 222, 틀린 개수: 18, 정답률: 92.5000%
전체: 240, 맞은 개수: 222, 틀린 개수: 18, 정답률: 92.5000%
전체: 240, 맞은 개수: 222, 틀린 개수: 18, 정답률: 92.5000%


In [129]:
for df in deepcopy(dfs):
    _, _ = evaluate_retriever(df, topk=4, is_preprocess=True)

전체: 240, 맞은 개수: 218, 틀린 개수: 22, 정답률: 90.8333%
전체: 240, 맞은 개수: 220, 틀린 개수: 20, 정답률: 91.6667%
전체: 240, 맞은 개수: 221, 틀린 개수: 19, 정답률: 92.0833%
전체: 240, 맞은 개수: 219, 틀린 개수: 21, 정답률: 91.2500%
전체: 240, 맞은 개수: 221, 틀린 개수: 19, 정답률: 92.0833%
전체: 240, 맞은 개수: 220, 틀린 개수: 20, 정답률: 91.6667%
전체: 240, 맞은 개수: 220, 틀린 개수: 20, 정답률: 91.6667%
전체: 240, 맞은 개수: 220, 틀린 개수: 20, 정답률: 91.6667%
전체: 240, 맞은 개수: 220, 틀린 개수: 20, 정답률: 91.6667%


In [130]:
for df in deepcopy(dfs):
    _, _ = evaluate_retriever(df, topk=3, is_preprocess=True)

전체: 240, 맞은 개수: 215, 틀린 개수: 25, 정답률: 89.5833%
전체: 240, 맞은 개수: 215, 틀린 개수: 25, 정답률: 89.5833%
전체: 240, 맞은 개수: 214, 틀린 개수: 26, 정답률: 89.1667%
전체: 240, 맞은 개수: 214, 틀린 개수: 26, 정답률: 89.1667%
전체: 240, 맞은 개수: 216, 틀린 개수: 24, 정답률: 90.0000%
전체: 240, 맞은 개수: 215, 틀린 개수: 25, 정답률: 89.5833%
전체: 240, 맞은 개수: 214, 틀린 개수: 26, 정답률: 89.1667%
전체: 240, 맞은 개수: 215, 틀린 개수: 25, 정답률: 89.5833%
전체: 240, 맞은 개수: 215, 틀린 개수: 25, 정답률: 89.5833%


In [131]:
for df in deepcopy(dfs):
    _, _ = evaluate_retriever(df, topk=2, is_preprocess=True)

전체: 240, 맞은 개수: 206, 틀린 개수: 34, 정답률: 85.8333%
전체: 240, 맞은 개수: 209, 틀린 개수: 31, 정답률: 87.0833%
전체: 240, 맞은 개수: 209, 틀린 개수: 31, 정답률: 87.0833%
전체: 240, 맞은 개수: 208, 틀린 개수: 32, 정답률: 86.6667%
전체: 240, 맞은 개수: 210, 틀린 개수: 30, 정답률: 87.5000%
전체: 240, 맞은 개수: 208, 틀린 개수: 32, 정답률: 86.6667%
전체: 240, 맞은 개수: 208, 틀린 개수: 32, 정답률: 86.6667%
전체: 240, 맞은 개수: 207, 틀린 개수: 33, 정답률: 86.2500%
전체: 240, 맞은 개수: 207, 틀린 개수: 33, 정답률: 86.2500%


In [132]:
for df in deepcopy(dfs):
    _, _ = evaluate_retriever(df, topk=1, is_preprocess=True)

전체: 240, 맞은 개수: 190, 틀린 개수: 50, 정답률: 79.1667%
전체: 240, 맞은 개수: 191, 틀린 개수: 49, 정답률: 79.5833%
전체: 240, 맞은 개수: 191, 틀린 개수: 49, 정답률: 79.5833%
전체: 240, 맞은 개수: 191, 틀린 개수: 49, 정답률: 79.5833%
전체: 240, 맞은 개수: 191, 틀린 개수: 49, 정답률: 79.5833%
전체: 240, 맞은 개수: 191, 틀린 개수: 49, 정답률: 79.5833%
전체: 240, 맞은 개수: 191, 틀린 개수: 49, 정답률: 79.5833%
전체: 240, 맞은 개수: 191, 틀린 개수: 49, 정답률: 79.5833%
전체: 240, 맞은 개수: 191, 틀린 개수: 49, 정답률: 79.5833%


## Rerank-BM25

In [27]:
bm25_path = os.path.join(parent_dir, "data", "bm25_train")

df_bm25_20 = pd.read_csv(os.path.join(bm25_path, "20_bm25_rerank_5.csv"))
df_bm25_30 = pd.read_csv(os.path.join(bm25_path, "30_bm25_rerank_5.csv"))
df_bm25_40 = pd.read_csv(os.path.join(bm25_path, "40_bm25_rerank_5.csv"))
df_bm25_50 = pd.read_csv(os.path.join(bm25_path, "50_bm25_rerank_5.csv"))
df_bm25_60 = pd.read_csv(os.path.join(bm25_path, "60_bm25_rerank_5.csv"))
df_bm25_70 = pd.read_csv(os.path.join(bm25_path, "70_bm25_rerank_5.csv"))
df_bm25_80 = pd.read_csv(os.path.join(bm25_path, "80_bm25_rerank_5.csv"))
df_bm25_90 = pd.read_csv(os.path.join(bm25_path, "90_bm25_rerank_5.csv"))
df_bm25_100 = pd.read_csv(os.path.join(bm25_path, "100_bm25_rerank_5.csv"))

dfs = [
    df_bm25_20,
    df_bm25_30,
    df_bm25_40,
    df_bm25_50,
    df_bm25_60,
    df_bm25_70,
    df_bm25_80,
    df_bm25_90,
    df_bm25_100,
]

In [28]:
for df in deepcopy(dfs):
    _, _ = evaluate_retriever(df)

전체: 240, 맞은 개수: 216, 틀린 개수: 24, 정답률: 90.0000%
전체: 240, 맞은 개수: 217, 틀린 개수: 23, 정답률: 90.4167%
전체: 240, 맞은 개수: 217, 틀린 개수: 23, 정답률: 90.4167%
전체: 240, 맞은 개수: 217, 틀린 개수: 23, 정답률: 90.4167%
전체: 240, 맞은 개수: 219, 틀린 개수: 21, 정답률: 91.2500%
전체: 240, 맞은 개수: 219, 틀린 개수: 21, 정답률: 91.2500%
전체: 240, 맞은 개수: 219, 틀린 개수: 21, 정답률: 91.2500%
전체: 240, 맞은 개수: 219, 틀린 개수: 21, 정답률: 91.2500%
전체: 240, 맞은 개수: 220, 틀린 개수: 20, 정답률: 91.6667%


In [30]:
for df in deepcopy(dfs):
    _, _ = evaluate_retriever(df, topk=4)

전체: 240, 맞은 개수: 214, 틀린 개수: 26, 정답률: 89.1667%
전체: 240, 맞은 개수: 216, 틀린 개수: 24, 정답률: 90.0000%
전체: 240, 맞은 개수: 216, 틀린 개수: 24, 정답률: 90.0000%
전체: 240, 맞은 개수: 216, 틀린 개수: 24, 정답률: 90.0000%
전체: 240, 맞은 개수: 218, 틀린 개수: 22, 정답률: 90.8333%
전체: 240, 맞은 개수: 218, 틀린 개수: 22, 정답률: 90.8333%
전체: 240, 맞은 개수: 217, 틀린 개수: 23, 정답률: 90.4167%
전체: 240, 맞은 개수: 217, 틀린 개수: 23, 정답률: 90.4167%
전체: 240, 맞은 개수: 217, 틀린 개수: 23, 정답률: 90.4167%


In [31]:
for df in deepcopy(dfs):
    _, _ = evaluate_retriever(df, topk=3)

전체: 240, 맞은 개수: 212, 틀린 개수: 28, 정답률: 88.3333%
전체: 240, 맞은 개수: 215, 틀린 개수: 25, 정답률: 89.5833%
전체: 240, 맞은 개수: 215, 틀린 개수: 25, 정답률: 89.5833%
전체: 240, 맞은 개수: 215, 틀린 개수: 25, 정답률: 89.5833%
전체: 240, 맞은 개수: 215, 틀린 개수: 25, 정답률: 89.5833%
전체: 240, 맞은 개수: 214, 틀린 개수: 26, 정답률: 89.1667%
전체: 240, 맞은 개수: 214, 틀린 개수: 26, 정답률: 89.1667%
전체: 240, 맞은 개수: 215, 틀린 개수: 25, 정답률: 89.5833%
전체: 240, 맞은 개수: 216, 틀린 개수: 24, 정답률: 90.0000%


In [32]:
for df in deepcopy(dfs):
    _, _ = evaluate_retriever(df, topk=2)

전체: 240, 맞은 개수: 206, 틀린 개수: 34, 정답률: 85.8333%
전체: 240, 맞은 개수: 207, 틀린 개수: 33, 정답률: 86.2500%
전체: 240, 맞은 개수: 206, 틀린 개수: 34, 정답률: 85.8333%
전체: 240, 맞은 개수: 205, 틀린 개수: 35, 정답률: 85.4167%
전체: 240, 맞은 개수: 207, 틀린 개수: 33, 정답률: 86.2500%
전체: 240, 맞은 개수: 207, 틀린 개수: 33, 정답률: 86.2500%
전체: 240, 맞은 개수: 207, 틀린 개수: 33, 정답률: 86.2500%
전체: 240, 맞은 개수: 208, 틀린 개수: 32, 정답률: 86.6667%
전체: 240, 맞은 개수: 208, 틀린 개수: 32, 정답률: 86.6667%


In [33]:
for df in deepcopy(dfs):
    _, _ = evaluate_retriever(df, topk=1)

전체: 240, 맞은 개수: 181, 틀린 개수: 59, 정답률: 75.4167%
전체: 240, 맞은 개수: 185, 틀린 개수: 55, 정답률: 77.0833%
전체: 240, 맞은 개수: 183, 틀린 개수: 57, 정답률: 76.2500%
전체: 240, 맞은 개수: 183, 틀린 개수: 57, 정답률: 76.2500%
전체: 240, 맞은 개수: 184, 틀린 개수: 56, 정답률: 76.6667%
전체: 240, 맞은 개수: 184, 틀린 개수: 56, 정답률: 76.6667%
전체: 240, 맞은 개수: 184, 틀린 개수: 56, 정답률: 76.6667%
전체: 240, 맞은 개수: 184, 틀린 개수: 56, 정답률: 76.6667%
전체: 240, 맞은 개수: 184, 틀린 개수: 56, 정답률: 76.6667%
